# Iris Prediction

We import the iris dataset, train and evaluate a XGBoost, and use this fitted model to make a prediction on a single observation.  
The objective is then to be able to make predictions on demand using Flask server and docker container.

### Imports

In [2]:
import numpy as np
import pandas as pd
import random
import pickle

import xgboost
import sklearn
from sklearn.datasets import load_iris

In [3]:
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(xgboost.__version__)

1.19.3
1.1.4
0.24.1
1.3.3


# Load Data

In [4]:
data = load_iris()
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [5]:
X, y = load_iris(return_X_y=True, as_frame=True)
df = pd.concat([X, y], axis=1)

In [13]:
df[df["target"]==1]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
55,5.7,2.8,4.5,1.3,1
56,6.3,3.3,4.7,1.6,1
57,4.9,2.4,3.3,1.0,1
58,6.6,2.9,4.6,1.3,1
59,5.2,2.7,3.9,1.4,1


In [14]:
sepal_length, sepal_width, petal_length, petal_width = 7.0, 3.2, 4.7, 1.4

In [15]:
"https://iris-plenitudeai.herokuapp.com//pred_args?sepal_length={}&sepal_width={}&petal_length={}&petal_width={}".format(sepal_length, sepal_width, petal_length, petal_width)

'https://iris-plenitudeai.herokuapp.com//pred_args?sepal_length=7.0&sepal_width=3.2&petal_length=4.7&petal_width=1.4'

In [8]:
"sepal_length={}&sepal_width={}&petal_length={}&petal_width={}".format(5.1, 3.5, 1.4, 0.2)

'sepal_length=5.1&sepal_width=3.5&petal_length=1.4&petal_width=0.2'

In [ ]:
5.1	3.5	1.4	0.2

### Train Test split

In [4]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df.drop("target", axis=1), df["target"])
X_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
83,6.0,2.7,5.1,1.6
145,6.7,3.0,5.2,2.3
143,6.8,3.2,5.9,2.3
47,4.6,3.2,1.4,0.2
67,5.8,2.7,4.1,1.0
...,...,...,...,...
55,5.7,2.8,4.5,1.3
7,5.0,3.4,1.5,0.2
54,6.5,2.8,4.6,1.5
121,5.6,2.8,4.9,2.0


# Modelling

### XGB

In [5]:
model_xgb = xgboost.sklearn.XGBClassifier()

### Train

In [6]:
model_xgb.fit(X_train, y_train)

[08:28:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\axeld\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
pred = model_xgb.predict(X_test)
pred_proba = model_xgb.predict_proba(X_test)

### Evaluation

We clearly overfit our data, but no problem this is not the interesting part of the project

In [8]:
sklearn.metrics.roc_auc_score(y_test, pred_proba, multi_class='ovr')

0.9959554334554334

# Serialization of model

In [9]:
with open("model.pkl", "wb") as file:
    pickle.dump(model_xgb, file)
    

# Prediction for 1 observation

In [10]:
with open("model.pkl", "rb") as file:
    model_loaded = pickle.load(file)

In [11]:
observation = X_test.sample(n=1)
observation

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
107,7.3,2.9,6.3,1.8


In [16]:
arg = list(observation.iloc[0])
arg

[7.3, 2.9, 6.3, 1.8]

In [18]:
pd.DataFrame([arg], columns=["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"])

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,7.3,2.9,6.3,1.8


In [12]:
pred_obs = model_loaded.predict(observation)
data.target_names[int(pred_obs)]

'virginica'